# Elixir: Sigils
* Sigils are one of the mechanisms provided by Elixir for using textual representations. Sigils start with the tilde (~) character which is followed by a letter (which identifies the sigil) and then a delimiter; modifiers can be added after the final delimiter.

### Regular expressions (the [regex module](https://hexdocs.pm/elixir/Regex.html))
* The most common sigil is __~r__, which creates regular expressions.

In [1]:
regex = ~r/foo|bar/

~r/foo|bar/

In [2]:
"foo" =~ regex

true

In [3]:
"bat" =~ regex

false

* Elixir provides Perl-compatible regular expressions, as implemented by the [PCRE library](http://www.pcre.org/). Regexes also support modifiers. For example, the i modifier makes a regular expression case insensitive.

In [4]:
"HELLO" =~ ~r/hello/

false

In [5]:
"HELLO" =~ ~r/hello/i

true

* Sigils support 8 different delimiters, not "/":
* The rationale is to provide a way to write literals without escaped delimiters. For example, a regular expression with forward slashes like ~r(^https?://) reads arguably better than ~r/^https?:\/\//. Similarly, if the regular expression has forward slashes and capturing groups (that use ()), you may then choose double quotes instead of parentheses.

### Strings
* The ~s sigil is used to generate strings, like double quotes. The ~s sigil is useful when a string contains double quotes.

In [6]:
~s(this is a string with "double" quotes, not 'single' ones)

"this is a string with \"double\" quotes, not 'single' ones"

### Char lists
* The ~c sigil is useful for generating char lists that contain single quotes:

In [7]:
~c(this is a char list containing 'single quotes')

'this is a char list containing \'single quotes\''

### Word lists
* The ~w sigil is used to generate lists of words (words are just regular strings). Inside the ~w sigil, words are separated by whitespace.
* The ~w sigil also accepts the c, s and a modifiers (for char lists, strings, and atoms, respectively), which specify the data type of the elements of the resulting list.

In [8]:
~w(foo bar bat)

["foo", "bar", "bat"]

In [9]:
~w(foo bar bat)a

[:foo, :bar, :bat]

### Interpolation & escaping in sigils
* Elixir also supports uppercase sigils to deal with escaping characters and interpolation. While both ~s and ~S will return strings, the former allows escape codes and interpolation. The latter does not.

In [10]:
~s(String with escape codes \x26 #{"inter" <> "polation"})

"String with escape codes & interpolation"

In [11]:
~S(String without escape codes \x26 without #{interpolation})

"String without escape codes \\x26 without \#{interpolation}"

The following escape codes can be used in strings and char lists:

\\ – single backslash
\a – bell/alert
\b – backspace
\d - delete
\e - escape
\f - form feed
\n – newline
\r – carriage return
\s – space
\t – tab
\v – vertical tab
\0 - null byte
\xDD - represents a single byte in hexadecimal (such as \x13)
\uDDDD and \u{D...} - represents a Unicode codepoint in hexadecimal (such as \u{1F600})

* A double quote inside a double-quoted string needs to be escaped as \".
* A single quote inside a single-quoted char list needs to be escaped as \'. 
* It is better style to change delimiters as seen above than to escape them.

### Heredocs
* Sigils also support heredocs, that is, triple double- or single-quotes as separators.
* The most common use case for heredoc sigils is writing documentation. For example, writing escape characters in the documentation would soon become error prone because of the need to double-escape some characters.

In [12]:
~s"""
this is a heredoc string
"""

"this is a heredoc string\n"

### Custom sigils
* Elixir sigils are extensible. Using the sigil ~r/foo/i is equivalent to calling sigil_r with a binary and a char list as the argument.

In [13]:
sigil_r(<<"foo">>, 'i')

~r/foo/i

* Accessing the documentation for the ~r sigil:

In [14]:
h sigil_r


                       defmacro sigil_r(term, modifiers)                        

Handles the sigil ~r for regular expressions.

It returns a regular expression pattern, unescaping characters and replacing
interpolations.

More information on regular expressions can be found in the Regex module.

## Examples

    iex> Regex.match?(~r(foo), "foo")
    true
    
    iex> Regex.match?(~r/abc/, "abc")
    true



* We can provide our own sigils by building functions that follow the sigil_{identifier} pattern.
* Let’s build the ~i sigil that returns an integer (with the optional n modifier to make it negative).

In [15]:
defmodule MySigils do
  def sigil_i(string, []), do: String.to_integer(string)
  def sigil_i(string, [?n]), do: -String.to_integer(string)
  end

{:module, MySigils, <<70, 79, 82, 49, 0, 0, 4, 156, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 150, 0, 0, 0, 15, 15, 69, 108, 105, 120, 105, 114, 46, 77, 121, 83, 105, 103, 105, 108, 115, 8, 95, 95, 105, 110, 102, 111, ...>>, {:sigil_i, 2}}

In [16]:
import MySigils

MySigils

In [17]:
~i(13)

13

In [18]:
~i(42)n

-42

* Sigils can be used to do compile-time work with the help of macros. 
* For example, regular expressions in Elixir are compiled into an efficient representation during compilation of the source code, therefore skipping this step at runtime. If you’re interested in the subject, we recommend you learn more about macros and check out how sigils are implemented in the Kernel module (where the sigil_* functions are defined).